In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch.helpers import bulk
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'])

In [2]:
ind = "network_weather_2-2015.12.10"

my_query = \
{
  "size": 0,
  "aggs": {      
        "type_counts": {
            "terms": { "field": "_type" },
            "aggs": {      
                "src_counts": {
                     "terms": { "field": "src" , "size": 10000},
                     "aggs": {      
                        "dest_counts": {
                        "terms": { "field": "dest" , "size": 10000}
                        }
                     }
                  }
              }
        }
  }
}

In [3]:
response = es.search(index=ind, body=my_query, request_timeout=600)

In [4]:
#print(response['aggregations']['type_counts']['buckets'])


In [8]:

types = response['aggregations']['type_counts']['buckets']

for t in types:
    print(t['key'])
   # if t['key'] != 'packet_loss_rate': continue
    for s in t['src_counts']['buckets']:
        for d in s['dest_counts']['buckets']:
            src=s['key']
            dest=d['key']
            print ('s: ',src,'d: ', dest, '\t docs:', d['doc_count'])
            myscroll={
            "size":100000,
#             "fields": ["timestamp", "delay_mean","throughput","packet_loss","src","dest"],
            "fields": ["timestamp"],
            "query":{
                "filtered": {
                    "query": {
                         "match_all": {} 
                        },
                        "filter" : { 
                            "bool" : {
                                "must" : [
                            {"term": {"_type": {"value": t['key'] } } },
                            {"term": {"src":   {"value": src } } },
                            {"term": {"dest":  {"value": dest } } }
                                ]
                            }

                        }
                }
            }
            ,"sort": [{ "timestamp": { "order": "asc" }}]
            }
            
            toDelete={}
#             oldres=0
            oldtime=""
            counter=0
            scroll = scan(es, query=myscroll, index=ind, scroll='120s')
            for res in scroll:
#                 if counter==0: print(res)
                if res['fields']['timestamp'][0]==oldtime:# and res['fields']['src'][0]==src and res['fields']['dest'][0]==dest and res['fields']['packet_loss'][0]==oldres:
                    toDelete[res['_id']]=res['_index']
#                     print(res['fields'])
#                 oldres=res['fields']['packet_loss'][0]
                oldtime=res['fields']['timestamp'][0]
                counter+=1
#                if counter>200:
#                    break
#                 if counter%100==0: 
                
            print ("records:", counter, '\tto delete:',len(toDelete))
            bulk_body = []
            for x in toDelete:
                bulk_body.append('{"delete": {"_id": "'+x+'", "_index":"'+toDelete[x]+'", "_type":"'+t['key']+'"}}')
#             print (bulk_body)
            if len(toDelete):
                es.bulk('\n'.join(bulk_body))

latency
s:  85.122.31.74 d:  163.1.5.210 	 docs: 4497
records: 40 	to delete: 0
s:  85.122.31.74 d:  144.92.180.75 	 docs: 4484
records: 48 	to delete: 0
s:  85.122.31.74 d:  129.15.40.231 	 docs: 4436
records: 28 	to delete: 0
s:  85.122.31.74 d:  134.158.103.10 	 docs: 4124
records: 124 	to delete: 0
s:  85.122.31.74 d:  147.213.204.112 	 docs: 3630
records: 25 	to delete: 0
s:  85.122.31.74 d:  139.184.80.18 	 docs: 3569
records: 42 	to delete: 0
s:  85.122.31.74 d:  138.253.60.82 	 docs: 3335
records: 21 	to delete: 0
s:  85.122.31.74 d:  169.228.130.40 	 docs: 3109
records: 42 	to delete: 0
s:  85.122.31.74 d:  193.58.172.56 	 docs: 1814
records: 100 	to delete: 0
s:  85.122.31.74 d:  81.180.86.64 	 docs: 1715
records: 183 	to delete: 0
s:  85.122.31.74 d:  109.105.124.86 	 docs: 1415
records: 208 	to delete: 0
s:  85.122.31.74 d:  144.16.111.26 	 docs: 1399
records: 55 	to delete: 0
s:  85.122.31.74 d:  192.12.15.26 	 docs: 1342
records: 58 	to delete: 0
s:  85.122.31.74 d:  193.